In [79]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from dataclasses import dataclass
import re
import time
import json

In [80]:
driver = webdriver.Firefox()

In [81]:
driver.get("https://www.imdb.com/pt/")
link1 = driver.find_element(By.XPATH, r'//*[@Id="imdbHeader-navDrawerOpen"]')
link1.click()
link_populares = driver.find_element(By.XPATH, '//a[span[text()="250 séries mais populares"]]')
link_populares.click()
lista = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul')
lista_series = lista.find_elements(By.TAG_NAME, "li")

In [82]:
@dataclass
class Serie:
    titulo: str
    ano: str
    classificacao: float
    pagina: str
    episodios: int
    elenco: list
    popularidade: int


In [83]:
len(lista_series)

75

In [85]:
series_parciais = []
for i in range(len(lista_series)):
    try:
        item = lista.find_elements(By.TAG_NAME, "li")[i]
        titulo = item.find_element(By.CLASS_NAME, "ipc-title__text").text
        ano = item.find_element(By.XPATH, "./div/div/div/div/div[2]/div[2]/span[1]").text
        avaliacao = item.find_element(By.CLASS_NAME, "ipc-rating-star--rating").text
        episodios = item.find_element(By.XPATH, "./div/div/div/div/div[2]/div[2]/span[2]").text
        link = item.find_element(By.CLASS_NAME, "ipc-title-link-wrapper").get_attribute("href")
        
        
        re_result = re.search(r'^\d\,\d', avaliacao)
        re_titulo = re.sub(r'[0-9.]', '', titulo)
        classificacao = float(re_result.group(0).replace(",", ".")) if re_result else None

     
        series_parciais.append(Serie(re_titulo, ano, classificacao, link, episodios, [], 0))
        print(f"Coletado: {re_titulo}")
    except Exception as e:
        print(f"Erro ao coletar série {i}: {e}")

Coletado:  Breaking Bad
Coletado:  Planeta Terra II
Coletado:  Planeta Terra
Coletado:  Irmãos de Guerra
Coletado:  Chernobyl
Coletado:  A Escuta
Coletado:  Avatar: A Lenda de Aang
Coletado:  Planeta Azul II
Coletado:  Família Soprano
Coletado:  Cosmos: Uma Odisseia do Espaço-Tempo
Coletado:  Cosmos
Coletado:  Nosso Planeta
Coletado:  Game of Thrones
Coletado:  Bluey
Coletado:  O Mundo em Guerra
Coletado:  Fullmetal Alchemist: Brotherhood
Coletado:  Vida
Coletado:  Rick e Morty
Coletado:  Arremesso Final
Coletado:  Além da Imaginação
Coletado:  Ataque dos Titãs
Coletado:  A Guerra do Vietnã
Coletado:  Sherlock
Coletado:  Batman: A Série Animada
Coletado:  Arcane
Coletado:  Better Call Saul
Coletado:  Vida de Escritório
Coletado:  Planeta Azul
Coletado:  Na Fazenda com Clarkson
Coletado:  Scam : The Harshad Mehta Story
Coletado:  Planeta Humano
Coletado:  Planeta Gelado
Coletado:  HUNTER×HUNTER
Coletado:  Only Fools and Horses
Coletado:  Firefly
Coletado:  The Civil War
Coletado:  Death

In [86]:

for serie in series_parciais:
    try:
        driver.get(serie.pagina)
        time.sleep(2)
        popularidade = driver.find_element(By.XPATH, "//*[@data-testid='hero-rating-bar__popularity__score']").text
        elenco_items = driver.find_elements(By.XPATH, "//*[@data-testid='title-cast-item']")
        lista_atores = []

        for ator in elenco_items:
            try:
                nome = ator.find_element(By.XPATH, ".//*[@data-testid='title-cast-item__actor']").text
                personagem = ator.find_element(By.XPATH, ".//*[@class='title-cast-item__characters-list']/ul/li/a").text
                lista_atores.append(personagem + "|" + nome)
            except:
                continue

        serie.elenco = lista_atores
        serie.popularidade = popularidade
        print("Adicionado elenco: " + serie.titulo)
    except Exception as e:
        print(f"Erro no elenco da série {serie.titulo}: {e}")


Adicionado elenco:  Breaking Bad
Adicionado elenco:  Planeta Terra II
Adicionado elenco:  Planeta Terra
Adicionado elenco:  Irmãos de Guerra
Adicionado elenco:  Chernobyl
Adicionado elenco:  A Escuta
Adicionado elenco:  Avatar: A Lenda de Aang
Adicionado elenco:  Planeta Azul II
Adicionado elenco:  Família Soprano
Adicionado elenco:  Cosmos: Uma Odisseia do Espaço-Tempo
Adicionado elenco:  Cosmos
Adicionado elenco:  Nosso Planeta
Adicionado elenco:  Game of Thrones
Adicionado elenco:  Bluey
Adicionado elenco:  O Mundo em Guerra
Adicionado elenco:  Fullmetal Alchemist: Brotherhood
Adicionado elenco:  Vida
Adicionado elenco:  Rick e Morty
Adicionado elenco:  Arremesso Final
Adicionado elenco:  Além da Imaginação
Adicionado elenco:  Ataque dos Titãs
Adicionado elenco:  A Guerra do Vietnã
Adicionado elenco:  Sherlock
Adicionado elenco:  Batman: A Série Animada
Adicionado elenco:  Arcane
Adicionado elenco:  Better Call Saul
Adicionado elenco:  Vida de Escritório
Erro no elenco da série  Pla

In [87]:
with open("Series/series.json","w",encoding="utf-8") as arquivo:
    for serie in series_parciais:
        json.dump(serie.__dict__,arquivo, ensure_ascii=False, indent=4)